###Coursera Capstone Project
###The Battle of Neighborhoods (Week 1)

In [3]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

**Coursera Capstone - REPORT CONTENT**

*  Section : ⁃ Discussion of the business problem and the interested audience in this project.

* Data Section: ⁃ Description of the data that will be used to solve the problem and the sources.

* Methodology section ⁃ Discussion and description of exploratory data analysis carried out, any inferential statistical testing performed,and if any machine learnings were used establishing the strategy and purposes.

* Results section ⁃ Discussion of the results.

* Discussion section ⁃ Elaboration and discussion on any observations noted and any recommendations suggested based on the results.

* Conclusion section ⁃ Report Conclusion.


**1. Introduction Section :**

Discussion of the business problem and the audience who would be interested in this project.
Description of the Problem and Background

**Scenario:**

I am a data scientist residing in India. I have been offered a great opportunity to work for a leader firm in Manhattan, NY. I am very excited and I want to use this opportunity to practice my learnings in Coursera in order to answer relevant questions arisen. The key question is : How can I find a convenient and enjoyable place similar to mine now in India? Certainly, I can use available real estate apps and Google but the idea is to use and apply myself the learned tools during the course. In order to make a comparison and evaluation of the rental options in Manhattan NY, I must set some basis, 

therefore the apartment in Manhattan must meet the following demands:

apartment must be 2 or 3 bedrooms
desired location is near a metro station in the Manhattan area and within 1.0 mile (1.6 km) radius
price of rent not exceed $7,000 per month
top ammenities in the selected neighborhood shall be similar to current residence
desirable to have venues such as coffee shops, restaurants Asian Thai, wine stores, gym and food shops
as a reference, I have included a map of venues near current residence in India.
Business Problem:
The challenge is to find a suitable apartment for rent in Manhattan NY that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.

**Interested Audience**

I believe this is a relevant challenge with valid questions for anyone moving to other large city in US, EU or Asia. The same methodology can be applied in accordance to demands as applicable. This case is also applicable for anyone interested in exploring starting or locating a new business in any city. Lastly, it can also serve as a good practical exercise to develop Data Science skills.

**2. Data Section:**
    
Description of the data and its sources that will be used to solve the problem
Description of the Data:¶
The following data is required to answer the issues of the problem:

* List of Boroughs and neighborhoods of Manhattan with their geodata (latitud and longitud)
* List of Subway metro stations in Manhattan with their address location
* List of apartments for rent in Manhattan area with their addresses and price
* Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc
* Venues for each Manhattan neighborhood ( than can be clustered)
* Venues for subway metro stations, as needed
* How the data will be used to solve the problem
* The data will be used as follows:

Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)
Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately
Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.
create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.
Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.
The procesing of these DATA will allow to answer the key questions to make a decision:

what is the cost of rent (per square ft) around a mile radius from each subway metro station?
what is the area of Manhattan with best rental pricing that meets criteria established?
What is the distance from work place ( Park Ave and 53 rd St) and the tentative future home?
What are the venues of the two best places to live? How the prices compare?
How venues distribute among Manhattan neighborhoods and around metro stations?
Are there tradeoffs between size and price and location?
Any other interesting statistical data findings of the real estate and overall data.
Reference of venues around current residence in India for comparison to Manhattan place

In [5]:
# Shenton Way, District 01, Singapore
address = 'Srinagar colony,Hyderabad, India'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of India home are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of India home are 17.431231, 78.4337951.


In [15]:
neighborhood_latitude =17.431231
neighborhood_longitude=78.4337951

In [16]:
CLIENT_ID = 'X1OVX2EVVIWJEKGCUWDZINMHFU5J103UXLZXHDBLTEHMMXGL' # your Foursquare ID
CLIENT_SECRET = 'YSX5VERZEH2SGVHPIZHOGLI0JOJRE04J5YCAHKD3Q1PVXD3Z' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X1OVX2EVVIWJEKGCUWDZINMHFU5J103UXLZXHDBLTEHMMXGL
CLIENT_SECRET:YSX5VERZEH2SGVHPIZHOGLI0JOJRE04J5YCAHKD3Q1PVXD3Z


In [29]:
LIMIT =  100  # limit of number of venues returned by Foursquare API# limit  
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=X1OVX2EVVIWJEKGCUWDZINMHFU5J103UXLZXHDBLTEHMMXGL&client_secret=YSX5VERZEH2SGVHPIZHOGLI0JOJRE04J5YCAHKD3Q1PVXD3Z&v=20180605&ll=17.431231,78.4337951&radius=1000&limit=100'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d09c6690d2be70039311726'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4cc1c709967d5481df0b7a57-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1c9941735',
         'name': 'Ice Cream Shop',
         'pluralName': 'Ice Cream Shops',
         'primary': True,
         'shortName': 'Ice Cream'}],
       'id': '4cc1c709967d5481df0b7a57',
       'location': {'address': 'Near Srinagar Colony',
        'cc': 'IN',
        'city': 'Hyderabad',
        'country': 'India',
        'distance': 687,
        'formattedAddress': ['Near Srinagar Colony',
         'Hyderabad 500073',
         'Telangana',
         'India'],
        'labeledLatLngs': [{'label': 'display

In [31]:

# function that extracts the category of the venue# functi 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(50)

,name,categories,lat,lng
0,Naturals Ice Cream,Ice Cream Shop,17.428536,78.439620
1,Park Hyatt Hyderabad,Hotel,17.424764,78.429600
2,Big Byte,Bakery,17.430177,78.438352
3,Q-Mart,Department Store,17.426681,78.438616
4,Poorna Tiffin Center,Breakfast Spot,17.433254,78.426191
5,Testa Rossa Caffe,Coffee Shop,17.424747,78.438038
6,Creamstone,Dessert Shop,17.432899,78.429327
7,Harley Davidson,Motorcycle Shop,17.425133,78.433314
8,Tre Forni,Italian Restaurant,17.424806,78.429600
9,WOFL,Breakfast Spot,17.424944,78.438148


Map of Hyderabad with venues near residence place - for reference

In [36]:
# create map of Singapore place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg